# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [4]:
import pandas as pd

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [36]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [7]:
df_filtered = df[(df['Total Claim Amount']>1000) & (df['Response'] == 'Yes')]
df_filtered.shape

(67, 26)

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [12]:
df_2 = df[df['Response'] == 'Yes']
df_2.groupby(['Policy','Gender'])['Total Claim Amount'].mean().sort_values()

Policy        Gender
Corporate L3  M         391.613774
Special L2    F         396.625189
Special L3    M         404.747560
Corporate L3  F         421.231615
Corporate L1  M         426.482338
Corporate L2  M         427.988391
Personal L2   F         439.905973
Corporate L2  F         441.345018
Personal L3   F         447.076355
              M         452.948421
Personal L2   M         459.430442
Corporate L1  F         463.942272
Personal L1   M         464.620881
Special L2    M         466.855864
Special L1    F         471.344049
              M         476.390378
Personal L1   F         485.125282
Special L3    F         495.566135
Name: Total Claim Amount, dtype: float64

In [ ]:
Conclusion = 'Los que más dinero el seguro tiene que pagar es a los vehiculos de policy Special L3 y con conductor mujer y el que menos Corporate L3 mujer'

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [35]:
df_3 = df.copy()
df_3_filtered = df_3.groupby(['State','Policy'])['Customer'].count().reset_index(name='count')
df_3_sorted = df_3_filtered[df_3_filtered['count']>500]
df_3_sorted

,State,Policy,count
5,Arizona,Personal L3,742
13,California,Personal L2,905
14,California,Personal L3,1250
31,Oregon,Personal L2,618
32,Oregon,Personal L3,1157


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [42]:
df_4 = df.copy()
df_4_grouped = df_4.groupby(['Education','Gender'])['Customer Lifetime Value'].agg(['count','max','min','median'])
round(df_4_grouped, 2)

count       max      min   median
Education            Gender                                   
Bachelor             F        1700  73225.96  1904.00  5640.51
                     M        1572  67907.27  1898.01  5548.03
College              F        1608  61850.19  1898.68  5623.61
                     M        1596  61134.68  1918.12  6005.85
Doctor               F         200  44856.11  2395.57  5332.46
                     M         201  32677.34  2267.60  5577.67
High School or Below F        1590  55277.45  2144.92  6039.55
                     M        1556  83325.38  1940.98  6286.73
Master               F         475  51016.07  2417.78  5729.86
                     M         412  50568.26  2272.31  5579.10

In [ ]:
Conclusion = "Las mujeres con educación básica tiene más valor, seguidos de los hombres con educación básica, las personas que tiene el mayor de los estudios que es el doctorado, son los que tienen el menos valor"

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [53]:
df_5 = df.copy()
df_5['Effective To Date'] = pd.to_datetime(df_5['Effective To Date'], format='%m/%d/%y')
df_5['Month'] = df_5['Effective To Date'].dt.month

In [58]:
df_5_grouped = df_5.pivot_table(index='State', columns='Month', values='Policy', aggfunc=['count'])
df_5_grouped

count      
Month          1     2
State                 
Arizona     1008   929
California  1918  1634
Nevada       551   442
Oregon      1565  1344
Washington   463   425

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [69]:
df_6 = df_5.copy()
df_6_grouped = df_6.groupby(['Month','State'])['Policy'].count().sort_values(ascending=False)[:3].reset_index()
df_6_grouped

,Month,State,Policy
0,1,California,1918
1,2,California,1634
2,1,Oregon,1565


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [71]:
df_7 = df_5.copy()
df_7_grouped = df_7.groupby(['Sales Channel'])['Response'].count()
df_7_grouped

Sales Channel
Agent          3890
Branch         2865
Call Center    2013
Web            1511
Name: Response, dtype: int64